In [ ]:
"""Plots ground truth and does 1KNN interpolation.
"""

In [ ]:
import sites_train_val_split as lib
import hsi_data

In [ ]:
import numpy as np

In [ ]:
%matplotlib ipympl

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import ipympl

In [ ]:
def apply_colormap(img, class_map):
    """
    class_map: list< list< int >( 3 ) >
        maps classes to colors
    """
    h, w = img.shape
    new_img = np.zeros((h*w,3))
    for i, p in enumerate(img.flatten()):
        if p == -1:
            new_img[i] = [0,0,0]
        else:
            new_img[i] = class_map[p]
    new_img = new_img / 255.
    return new_img.reshape((h,w,3))
        

In [ ]:
pavia_class_map = [[192, 192, 192], [41, 253, 47], [45, 255, 253], [14, 126, 18], [250, 40, 251], [162, 81, 43], [126, 11, 123], [247, 9, 27], [254, 253, 56] ]

In [ ]:
dataset = 'PaviaU'


In [ ]:
trainimgname, trainlabelname = hsi_data.dset_filenames_dict[dataset]
trainimgfield, trainlabelfield = hsi_data.dset_fieldnames_dict[dataset]
labels = hsi_data.load_labels(trainlabelname, trainlabelfield)

h,w,b = hsi_data.dset_dims[trainimgname]
nclass = hsi_data.nclass_dict[dataset]

class_to_components = lib.get_labeled_sites(labels)

In [ ]:

plt.figure()
test_mask_im = np.zeros(hsi_data.dset_dims['KSC_corrected.mat'][:2])

for compi, comp in enumerate(class_to_components[6]):
    for i, j in comp:
        test_mask_im[i,j]=compi+1

plt.imshow(test_mask_im)

In [ ]:
len(class_to_components[7])

In [ ]:
plt.figure()
test_mask_im = np.zeros(hsi_data.dset_dims['KSC_corrected.mat'][:2])

comp = [(210, 589), (210, 590), (211, 588), (211, 589), (211, 590), (210, 591), (210, 592), (210, 593), (214, 584), (215, 584), (214, 583), (215, 583), (216, 583), (216, 584), (217, 591), (217, 592), (217, 593), (217, 594), (217, 595), (218, 587), (218, 588), (218, 589), (218, 590), (218, 591), (218, 592), (218, 593), (218, 594), (218, 595), (219, 589), (219, 590), (219, 591), (219, 592), (219, 593), (219, 594), (220, 590), (220, 591), (220, 592), (220, 593)]
for i, j in comp:
    test_mask_im[i,j]=compi+1

plt.imshow(test_mask_im)

In [ ]:
labels.shape

In [ ]:
mask_path = '/scratch0/ilya/locDoc/data/hyperspec/KSC_strictsinglesite_trainval_s50_9_307364.mat'
train_mask = hsi_data.multiversion_matfile_get_field(mask_path, 'train_mask')
train_mask_im = train_mask.reshape(hsi_data.dset_dims['KSC_corrected.mat'][:2][::-1]).T

In [ ]:
mask_path = '/scratch0/ilya/locDoc/data/hyperspec/PaviaU_strictsinglesite_trainval_s90_9_484519.mat'
# mask_path = '/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/pyfst/masks/PaviaU_distributed_trainval_p0200_0_112681.mat'
train_mask = hsi_data.multiversion_matfile_get_field(mask_path, 'train_mask')
train_mask_im = train_mask.reshape(hsi_data.dset_dims['PaviaU.mat'][:2][::-1]).T

In [ ]:
mask_path = '/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/pyfst/masks/IP_distributed_trainval_s10_0_492018.mat'
train_mask = hsi_data.multiversion_matfile_get_field(mask_path, 'train_mask')
train_mask_im = train_mask.reshape(hsi_data.dset_dims['Indian_pines_corrected.mat'][:2][::-1]).T

In [ ]:
mask_path = '/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/pyfst/masks/Botswana_strictsinglesite_trainval_s05_7_647323.mat'
train_mask = hsi_data.multiversion_matfile_get_field(mask_path, 'train_mask')
train_mask_im = train_mask.reshape(hsi_data.dset_dims['Botswana.mat'][:2][::-1]).T

In [ ]:
pav_im = (labels * train_mask_im).astype(int)-1

In [ ]:
fig = plt.figure()
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
# ax.imshow(apply_colormap(pav_im, pavia_class_map))
# ax.imshow(apply_colormap(img_pred-1, pavia_class_map))
ax.imshow(apply_colormap(labels.astype(int)-1, pavia_class_map))

In [ ]:
plt.close('all')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
trainX = []
trainY = []
testX = []
testY = []
wholeImg = []
for i in range(labels.shape[0]):
    for j in range(labels.shape[1]):
        if train_mask_im[i,j]:
            trainX.append([i,j])
            trainY.append(labels[i,j])
        elif int(labels[i,j]):
            testX.append([i,j])
            testY.append(labels[i,j])
        wholeImg.append([i,j])

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
classifier.fit(trainX, trainY)

In [ ]:
y_pred = classifier.predict(wholeImg)

In [ ]:
img_pred = np.array(y_pred).reshape(hsi_data.dset_dims['PaviaU.mat'][:2])

In [ ]:
y_pred_ = classifier.predict(testX)
(np.array(testY) == np.array(y_pred_)).sum() / float(len(testY))

In [ ]:
len(testY)